# **Extracción, Transformación y Carga (ETL) - Dataset: Telefonía Móvil**

Se hará el proceso de ETL para el dataset "Telefonía Móvil" del proyecto de Data Analyst en telecomunicaciones. Se inicia priorizando cuáles de las 7 hojas del dataset son relevantes para el proyecto.

## **1. Hojas relevantes**

Luego de una revisión se considera que para el fin del proyecto, el cual es la realización de un análisis completo que permita reconocer el comportamiento del sector de las telecomunicaciones a nivel nacional, solamente se tendrán en cuentas algunas páginas de el dataset de "Telefonía móvil".

* **Accesos:** Esta hoja es útil porque muestra la cantidad de accesos pospago y prepago, lo que nos podría dar una idea de la penetración del servicio en la población. Esto permitiría entender cuántos usuarios utilizan los servicios móviles, una métrica clave para evaluar el crecimiento del mercado.

* **Minutos salientes:** El análisis de los minutos de llamadas salientes permitiría observar los patrones de consumo y uso del servicio móvil. Esto permitirá detectar tendencias en el uso del servicio de llamadas y cómo han cambiado con el tiempo, particularmente entre usuarios de pospago y prepago.

* **Ingresos:** Los ingresos generados por los servicios de telefonía móvil son una métrica clave para entender el impacto económico del sector. Esto nos permitirá correlacionar el uso de los servicios (llamadas, SMS, accesos) con los ingresos generados, proporcionando una visión clara del rendimiento financiero del sector.

## **2. Carga de la base de datos**

In [ ]:
import pandas as pd

# Cargamos el dataset
datos = '/content/telefonia_movil.xlsx'

In [ ]:
# Seleccionamos las hojas
hojas = ['Accesos', 'Minutos salientes', 'Ingresos']

In [ ]:
# Cargamos cada hoja en un diccionario de DataFrames
dataframes_hojas = {hoja: pd.read_excel(datos, sheet_name=hoja) for hoja in hojas}

In [ ]:
# Exploramos la estructura de los datos cargados
for hoja, df in dataframes_hojas.items():
    print("###################################################################")
    print(f"Hoja: {hoja}")
    print("###################################################################")
    print(df.info())
    print("###################################################################")
    print(df.head())

###################################################################
Hoja: Accesos
###################################################################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Año                          45 non-null     int64  
 1   Trimestre                    45 non-null     int64  
 2   Total de accesos pospago     45 non-null     float64
 3   Total de accesos prepago     45 non-null     float64
 4   Total de accesos operativos  45 non-null     int64  
 5   Periodo                      45 non-null     object 
dtypes: float64(2), int64(3), object(1)
memory usage: 2.2+ KB
None
###################################################################
    Año  Trimestre  Total de accesos pospago  Total de accesos prepago  \
0  2013          1                 7761349.0                57147608.0   
1

## **3. Datos faltantes**

Procedemos a identificar las columnas con valores faltantes y decidir cómo tratarlos (imputación, eliminación, etc.).

In [ ]:
# Verificamos los datos faltantes en cada hoja
for hoja, df in dataframes_hojas.items():
    print(f"Datos faltantes en la hoja: {hoja}")
    print(df.isnull().sum())
    print("###################################################################")

Datos faltantes en la hoja: Accesos
Año                            0
Trimestre                      0
Total de accesos pospago       0
Total de accesos prepago       0
Total de accesos operativos    0
Periodo                        0
dtype: int64
###################################################################
Datos faltantes en la hoja: Minutos salientes
Año                                   0
Trimestre                             0
Minutos pospago salientes (miles)     0
Minutos prepago salientes (miles)     0
Total de minutos salientes (miles)    0
Periodo                               0
dtype: int64
###################################################################
Datos faltantes en la hoja: Ingresos
Año                      0
Trimestre                0
Ingresos (miles de $)    0
Periodo                  0
dtype: int64
###################################################################


Como podemos apreciar, no tenemos datos faltantes en ninguna de las tres hojas del dataset.

## **4. Datos duplicados**

Primero identificaremos si existen datos duplicados en cada hoja. En caso de encontralos procederemos a eliminarlos.

In [ ]:
# Verificamos y eliminamos duplicados en cada hoja
for hoja, df in dataframes_hojas.items():
    print(f"Antes de eliminar duplicados en {hoja}: {df.duplicated().sum()} filas duplicadas")

    # Eliminar duplicados
    df.drop_duplicates(inplace=True)

    print(f"Después de eliminar duplicados en {hoja}: {df.duplicated().sum()} filas duplicadas")
    print("###################################################################")

Antes de eliminar duplicados en Accesos: 0 filas duplicadas
Después de eliminar duplicados en Accesos: 0 filas duplicadas
###################################################################
Antes de eliminar duplicados en Minutos salientes: 0 filas duplicadas
Después de eliminar duplicados en Minutos salientes: 0 filas duplicadas
###################################################################
Antes de eliminar duplicados en Ingresos: 0 filas duplicadas
Después de eliminar duplicados en Ingresos: 0 filas duplicadas
###################################################################


Como podemos apreciar, no habían datos duplicados.

## **5. Outliers**

Usaremos el método del rango intercuartílico (IQR) para identificar posibles outliers en las columnas numéricas.

El método del rango intercuartílico (IQR) es una técnica comúnmente utilizada para identificar valores atípicos (outliers) en columnas numéricas. El $IQR$ es la diferencia entre el tercer cuartil ($Q_3$) y el primer cuartil ($Q_1$) de los datos. Los cuartiles dividen los datos en partes iguales, donde:

*   $Q_1$: Es el valor que deja por debajo el $25\%$ de los datos.
*   $Q_3$: Es el valor que deja por debajo el $75\%$ de los datos.
*   $IQR$ (rango intercuartílico): Es la diferencia entre el tercer y el primer cuartil. Representa la amplitud de la "zona central" de los datos, es decir, el rango donde se encuentra el $50\%$ central de los datos.

In [ ]:
# Creamos una función para detectar outliers usando el método del IQR
def detecta_outliers(df, column):
    """
    Detecta outliers en una columna numérica de un DataFrame usando el método del rango intercuartílico (IQR).

    Parámetros:
    df (pandas.DataFrame): El DataFrame que contiene los datos.
    column (str): El nombre de la columna en la cual se quieren detectar los outliers.

    Retorna:
    pandas.DataFrame: Un DataFrame que contiene únicamente las filas donde los valores de la columna son outliers.

    El método del IQR identifica outliers como aquellos valores que están por debajo de Q1 - 1.5 * IQR o
    por encima de Q3 + 1.5 * IQR, donde Q1 es el primer cuartil (25%) y Q3 es el tercer cuartil (75%).
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

In [ ]:
# Detectamos outliers en las columnas numéricas de cada hoja
for hoja, df in dataframes_hojas.items():
    print(f"Hoja: {hoja}")
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    for col in numeric_cols:
        outliers = detecta_outliers(df, col)
        print(f"Columna: {col}, Outliers encontrados: {len(outliers)}")
        if len(outliers) > 0:
            print(outliers.head())  # Mostrar algunos outliers para inspección
    print("###################################################################")


Hoja: Accesos
Columna: Año, Outliers encontrados: 0
Columna: Trimestre, Outliers encontrados: 0
Columna: Total de accesos pospago, Outliers encontrados: 0
Columna: Total de accesos prepago, Outliers encontrados: 2
    Año  Trimestre  Total de accesos pospago  Total de accesos prepago  \
2  2013          3                 8002183.0                59048909.0   
3  2013          4                 8674516.0                58894516.0   

   Total de accesos operativos        Periodo  
2                     67051092  Jul-Sept 2013  
3                     67569032   Oct-Dic 2013  
Columna: Total de accesos operativos, Outliers encontrados: 0
###################################################################
Hoja: Minutos salientes
Columna: Año, Outliers encontrados: 0
Columna: Trimestre, Outliers encontrados: 0
Columna: Minutos pospago salientes (miles), Outliers encontrados: 0
Columna: Minutos prepago salientes (miles), Outliers encontrados: 5
     Año  Trimestre  Minutos pospago salientes 

### 5.1. Hoja `Accesos`

En la columna `Total de accesos prepago`, tenemos 2 registros en los trimestres 3 y 4 de 2013, con valores muy altos de accesos prepago. Es posible que estos valores reflejen un aumento real en el número de usuarios de prepago durante esos trimestres. Los valores atípicos podrían deberse a campañas comerciales o eventos que promovieron un crecimiento en el uso del prepago. No parecen ser errores de datos, ya que están dentro de un rango que podría ser plausible.

### 5.2. Hoja `Minutos salientes`

En la columna `Minutos prepago salientes (miles)`, tenemos 5 registros outliers entre 2022 y 2023. Estos son valores más bajos de minutos prepago salientes (comparados con otros trimestres) que podrían reflejar una tendencia decreciente en el uso de servicios de prepago para llamadas de voz.

### 5.3. Hoja `Ingresos`

En la columna `Ingresos (miles de $)`, tenemos 5 registros outliers que corresponden a trimestres recientes (2023-2024), con ingresos significativamente más altos. Estos valores pueden reflejar un aumento real en los ingresos, ya que en los últimos años ha habido un crecimiento importante en la industria de las telecomunicaciones debido al mayor consumo de servicios móviles y la adopción de nuevas tecnologías.

Dado que los outliers encontrados no parecen ser errores de datos, sino que más bien reflejan tendencias o diferencias reales en los trimestres analizados, estos de mantendrán para el análisis.

## **6. Transformación de los datos**

### 6.1. Revisión de formatos de fechas

Para poder verificar si tenemos columnas con información de fechas en todas las hojas del dataset, podemos buscar las columnas que contengan datos relacionados con fechas (en este caso, aquellas que podrían contener el año, mes o periodo) y revisar si están en el formato correcto. Para ello creamos la siguiente función

In [ ]:
# Función para identificar columnas que podrían contener fechas
def revisar_columnas_fechas(df):
    """
    Identifica columnas en un DataFrame que potencialmente contienen fechas basándose en los nombres de las columnas.

    Parámetros:
    df (pandas.DataFrame): El DataFrame que contiene los datos.

    Retorna:
    dict: Un diccionario donde las claves son los nombres de las columnas potencialmente relacionadas con fechas,
    y los valores indican si el formato es correcto (es decir, ya es un formato de fecha) o si necesita conversión.

    El criterio para identificar columnas de fechas es revisar si en el nombre de la columna aparece 'fecha', 'periodo'
    o 'año'. Luego, se intenta convertir los valores de esas columnas al formato datetime para verificar su validez.
    """
    # Identificar columnas que potencialmente contengan fechas: aquellas con 'fecha', 'periodo' o 'año' en su nombre
    columnas_potenciales_fechas = [col for col in df.columns if 'fecha' in col.lower() or 'periodo' in col.lower() or 'año' in col.lower()]

    # Verificar si las columnas ya están en formato de fecha o si necesitan conversión
    verificacion_fechas = {}
    for columna in columnas_potenciales_fechas:
        # Intentar convertir a datetime y verificar si hay errores
        try:
            pd.to_datetime(df[columna], errors='raise')
            verificacion_fechas[columna] = 'Formato de fecha correcto'
        except:
            verificacion_fechas[columna] = 'Formato incorrecto o texto'

    return verificacion_fechas

In [ ]:
# Verificamos todas las hojas para identificar posibles columnas de fechas
for hoja, df in dataframes_hojas.items():
    print(f"Verificando fechas en la hoja: {hoja}")
    result = revisar_columnas_fechas(df)  # Usamos la función que creamos
    if result:
        print(result)
    else:
        print("No se encontraron columnas relacionadas con fechas.")
    print("###################################################################")

Verificando fechas en la hoja: Accesos
{'Año': 'Formato de fecha correcto', 'Periodo': 'Formato incorrecto o texto'}
###################################################################
Verificando fechas en la hoja: Minutos salientes
{'Año': 'Formato de fecha correcto', 'Periodo': 'Formato incorrecto o texto'}
###################################################################
Verificando fechas en la hoja: Ingresos
{'Año': 'Formato de fecha correcto', 'Periodo': 'Formato incorrecto o texto'}
###################################################################


<ipython-input-16-c387c5440ad9>:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df[columna], errors='raise')
<ipython-input-16-c387c5440ad9>:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df[columna], errors='raise')
<ipython-input-16-c387c5440ad9>:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df[columna], errors='raise')


En todas las hojas, la columna `Periodo` contiene información de fechas como texto. Vamos a convertirla a un formato de fecha adecuado:

In [ ]:
# Revisamos el formato de fecha que tiene periodo
dataframes_hojas['Accesos']['Periodo'].head()

,Periodo
0,Ene-Mar 2013
1,Abr-Jun 2013
2,Jul-Sept 2013
3,Oct-Dic 2013
4,Ene-Mar 2013


In [ ]:
dataframes_hojas['Minutos salientes']['Periodo'].head()

,Periodo
0,Ene-Mar 2013
1,Abr-Jun 2013
2,Jul-Sept 2013
3,Oct-Dic 2013
4,Ene-Mar 2013


In [ ]:
dataframes_hojas['Ingresos']['Periodo'].head()

,Periodo
0,Ene-Mar 2013
1,Abr-Jun 2013
2,Jul-Sept 2013
3,Oct-Dic 2013
4,Ene-Mar 2013


In [ ]:
# Creamos una función para mapear los meses de texto a un rango adecuado de fechas (trimestres)
def convertir_periodo_a_fecha(periodo_str):
    """
    Convierte una cadena con formato de periodo (Ej: "Ene-Mar 2023") a una fecha.

    Parámetros:
    periodo_str (str): El periodo en formato texto.

    Retorna:
    pandas.Timestamp: La fecha convertida al último día del trimestre correspondiente.
    """
    # Definimos un diccionario para convertir trimestres a fechas
    trimestre_a_fecha = {
        'Ene-Mar': '03-31',
        'Abr-Jun': '06-30',
        'Jul-Sept': '09-30',
        'Oct-Dic': '12-31'
    }

    # Dividir el texto en mes y año
    for trimestre, fecha in trimestre_a_fecha.items():
        if trimestre in periodo_str:
            # Extraer el año
            year = periodo_str.split()[-1]
            # Construir la fecha completa
            return pd.to_datetime(f"{year}-{fecha}", format="%Y-%m-%d", errors='coerce')
    return pd.NaT # Si no se reconoce el formato, devolver NaT (nulo)

In [ ]:
# Aplicamos la conversión a la columna 'Periodo' en todas las hojas relevantes
for hoja, df in dataframes_hojas.items():
    if 'Periodo' in df.columns:
        df['Periodo'] = df['Periodo'].apply(convertir_periodo_a_fecha)
        print(f"Hoja: {hoja} - Columna 'Periodo' convertida con éxito.")
        print(df[['Periodo']].head())  # Verificamos los primeros valores convertidos
    print("###################################################################")

Hoja: Accesos - Columna 'Periodo' convertida con éxito.
     Periodo
0 2013-03-31
1 2013-06-30
2 2013-09-30
3 2013-12-31
4 2013-03-31
###################################################################
Hoja: Minutos salientes - Columna 'Periodo' convertida con éxito.
     Periodo
0 2013-03-31
1 2013-06-30
2 2013-09-30
3 2013-12-31
4 2013-03-31
###################################################################
Hoja: Ingresos - Columna 'Periodo' convertida con éxito.
     Periodo
0 2013-03-31
1 2013-06-30
2 2013-09-30
3 2013-12-31
4 2013-03-31
###################################################################


### 6.2. Normalización de los nombres de las columas

Vamos a normalizar los nombres de columnas para evitar problemas en el análisis posterior evitando espacios o caracteres especiales.

In [ ]:
# Creamos una función para normalizar nombres de columnas
def normaliza_nombres_columnas(df):
    """
    Normaliza los nombres de las columnas en un DataFrame.

    Esta función realiza los siguientes pasos en los nombres de las columnas:
    - Elimina espacios en blanco al inicio y final.
    - Convierte todas las letras a minúsculas.
    - Reemplaza los espacios entre palabras con guiones bajos.
    - Elimina caracteres no alfanuméricos.

    Parámetros:
    df (pandas.DataFrame): El DataFrame cuyas columnas se van a normalizar.

    Retorna:
    pandas.DataFrame: El DataFrame con los nombres de columnas normalizados.
    """
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(r'[^\w]', '', regex=True)
    return df

In [ ]:
# Aplicamos la normalización a todas las hojas
for hoja, df in dataframes_hojas.items():
    normaliza_nombres_columnas(df)

In [ ]:
# Verificamos algunos resultados
for hoja, df in dataframes_hojas.items():
    print(f"Nombres de columnas en la hoja {hoja}:")
    print(df.columns)
    print("###################################################################")

Nombres de columnas en la hoja Accesos:
Index(['año', 'trimestre', 'total_de_accesos_pospago',
       'total_de_accesos_prepago', 'total_de_accesos_operativos', 'periodo'],
      dtype='object')
###################################################################
Nombres de columnas en la hoja Minutos salientes:
Index(['año', 'trimestre', 'minutos_pospago_salientes_miles',
       'minutos_prepago_salientes_miles', 'total_de_minutos_salientes_miles',
       'periodo'],
      dtype='object')
###################################################################
Nombres de columnas en la hoja Ingresos:
Index(['año', 'trimestre', 'ingresos_miles_de_', 'periodo'], dtype='object')
###################################################################


Como podemos apreciar los nombres de las columnas han sido normalizados correctamente, eliminando espacios, convirtiendo a minúsculas y reemplazando caracteres no alfanuméricos.

### 6.3. Formato correcto en las columnas numéricas

Vamos a revisar si las columnas que deberían contener valores numéricos están en el formato adecuado (`int` o `float`). Si encontramos alguna columna numérica que esté en formato de texto o que tenga valores no numéricos, podemos intentar convertirla al formato correcto.

In [ ]:
# Hacemos un listado de las columnas que deben permanecer como texto en cada hoja
columnas_texto = {
    'Accesos': ['periodo'],  # 'Periodo' es texto pero ya lo convertimos a fecha, el resto debería ser numérico
    'Minutos salientes': ['periodo'],  # Igual para esta hoja
    'Ingresos': ['periodo']  # Solo 'Periodo' debe quedar como fecha en esta hoja
}

In [ ]:
# Revisamos y convertimos las columnas numéricas a int o float si es necesario
for hoja, df in dataframes_hojas.items():
    print(f"Revisando tipos de datos en la hoja: {hoja}")

    # Identificamos las columnas que deberían ser numéricas y no están en la lista de texto
    columnas_a_convertir = df.select_dtypes(include=['object']).columns.difference(columnas_texto.get(hoja, []))

    for col in columnas_a_convertir:
        try:
            # Intentamos convertir la columna a numérico (int o float)
            df[col] = pd.to_numeric(df[col], errors='coerce')
            print(f"Columna {col} convertida exitosamente a formato numérico.")
        except Exception as e:
            print(f"Error al convertir la columna {col}: {e}")

    # Verificamos el tipo de dato final de cada columna
    print("Tipos de datos finales:")
    print(df.dtypes)
    print("###################################################################")

Revisando tipos de datos en la hoja: Accesos
Tipos de datos finales:
año                                     int64
trimestre                               int64
total_de_accesos_pospago              float64
total_de_accesos_prepago              float64
total_de_accesos_operativos             int64
periodo                        datetime64[ns]
dtype: object
###################################################################
Revisando tipos de datos en la hoja: Minutos salientes
Tipos de datos finales:
año                                          int64
trimestre                                    int64
minutos_pospago_salientes_miles            float64
minutos_prepago_salientes_miles            float64
total_de_minutos_salientes_miles           float64
periodo                             datetime64[ns]
dtype: object
###################################################################
Revisando tipos de datos en la hoja: Ingresos
Tipos de datos finales:
año                            int64

Ahora las columnas están correctamente formateadas:

* **Accesos:** Las columnas numéricas (`total_de_accesos_pospago`, `total_de_accesos_prepago`, etc.) están en formato `float64`, excepto `año`, `trimestre`, y `total_de_accesos_operativos`, que son enteros.

* **Minutos salientes:** Las columnas numéricas también están en el formato correcto (`float64`), y la columna periodo está correctamente en formato de fecha.

* **Ingresos:** Las columnas numéricas (`ingresos_miles_de_`) están en formato `int64`, y la columna `periodo` está en formato de fecha.

Con estas pasos, hemos finalizado la etapa de transformación de datos, asegurándonos de que todos los valores estén en los formatos adecuados. Ahora los datos están completamente preparados para el análisis.

In [ ]:
# Crea un archivo ExcelWriter para guardar los DataFrames en diferentes hojas
#with pd.ExcelWriter('dataframes_hojas.xlsx') as writer:
  #for sheet_name, df in dataframes_hojas.items():
    #df.to_excel(writer, sheet_name=sheet_name, index=False)
#files.download('dataframes_hojas.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>